In [19]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

## Run Data Prep (JSON annotations), Train, and Eval

In [ ]:
%cd /Users/rebekaheichberg/Desktop/git-re/erdos-2025-colorectal-imaging
%env PYTHONPATH=src:$PYTHONPATH

!python src/faster_rcnn/scripts/train_eval_faster_rcnn.py \
    --datasets testing_faster \
    --images-root testing_faster/images/val \
    --epochs 1


## Run with Exisiting JSON annotations, Train, and Eval

In [ ]:
%cd /Users/rebekaheichberg/Desktop/git-re/erdos-2025-colorectal-imaging
%env PYTHONPATH=src:$PYTHONPATH

!python src/faster_rcnn/scripts/train_eval_faster_rcnn.py \
    --datasets testing_faster \
    --images-root testing_faster/images/val \
    --epochs 1 \
    --skip-prepare